In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward(ctx, x -> 2.0*x + x, Params(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 42
ctx = ⬅Context()
ForwardBackward(ctx, x -> x^2.0 + x^2.0, Params(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), Utero.Params(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = Params(ctx, 13.2, 89.0)
ForwardBackward(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

# Differentiation for Scalar Parameters

In [6]:
x = 12 # treated as constant 

Params = IdDict()
w = Utero.Tracked(7.0, Params)
grads = Utero.GetGradient(_ -> 2 * (((2*w + x) - w) * w) + sin(w^2), Params) 
# should be approx. 56.21
println(grads)

Params = IdDict()
w = Utero.Tracked(25.0, Params)
grads = Utero.GetGradient(_ -> sin(w)^2, Params)
# should be approx. -0.26
println(grads)

Params = IdDict()
w1 = Utero.Tracked(13.2, Params); w2 = Utero.Tracked(89.0, Params)
grads = Utero.GetGradient((x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 
# should be approx. (27.53, 3.0428)
println(grads)


ErrorException: cannot assign a value to imported variable Utero.Params from module Main

# Differentiation for Tensor Parameters

In [7]:

Parameters = IdDict()
w1 = Utero.Tracked(rand(2, 2), Parameters)
w2 = Utero.Tracked(rand(2, 2), Parameters)
w3 = Utero.Tracked(rand(2, 2), Parameters)

grads = Utero.GetGradient(x -> prod(*(w3, *(w2, *(w1, x)))), rand(2,1), Parameters)

UndefVarError: UndefVarError: `Tracked` not defined

The product of the parameter matrices should match A.

In [8]:
A = rand(2,2)
X = rand(100, 2)
Y = mapslices(v -> (A * v), X, dims=(2))


println(A)
println((w1 * w2 * w3).val)

for (x, y) in zip(eachrow(X), eachrow(Y))
    x = vec(x); y = vec(y)
    grads = Utero.GetGradient(x -> Utero.mse(*(w3, *(w2, *(w1, x))), y), x, Parameters)
    Utero.GradientStep(0.01, Parameters, grads)
end

println("Now, they should match")
println(A)
println((w1 * w2 * w3).val)

[0.5962591746131758 0.06944860965323862; 0.6601123287469913 0.620802535031863]


UndefVarError: UndefVarError: `w1` not defined